# Analysing Stock Prices

## Introduction to Stock Price Data

In this project, we will practice building a linked-list data structure by replicating Python's built-in `dict` class.

We will work with stock market data that was downloaded from [Yahoo Finance](https://finance.yahoo.com/?guccounter=1&guce_referrer=aHR0cHM6Ly9hcHAuZGF0YXF1ZXN0LmlvLw&guce_referrer_sig=AQAAACKkA8nSMrgBDleCbgIRemSlGPHG4PxKiHEZmoASJcATGOTE6RWhAUkIJbcwOJ6YHcFMwxZHOta2KqdfmZnzn3_URTLKLynTd4_PDalhMrfvNsyPYH3y2Ky-rh-jlkGDf-ARM8n1yzACxQ5mDv5VY8-mExzW8VSJTyuMSumKlce-) using the [yahoo_finance](https://pypi.org/project/yahoo-finance/) Python package.
This data consists of the daily stock prices from `2007-1-11` to `2017-04-17` for several hundred stock symbols traded on the [NASDAQ](https://www.nasdaq.com/) stock exchange, stored in the `prices` folder. The `download_data.py` script in the main project directory was used to download off ol the stock price data.

Each file in the `prices` folder is names for a specific stock symbol and contains the following columns:

* `date`: date that the data is from.
* `close`: the closing price on that day, which is the price when the trading day ends.
* `open`: the opening price on that day, which is the price when the trading day starts.
* `high`: the highest price the stock reached during trading.
* `low`: the lowest price the stock reached during trading.
* `volume`: the number of shares that were traded during the day.

First, let's create a `Dictionary` class to allow us to store and explore our data.


In [1]:
# Implement a node class
class Node:
    
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None
        
# Test the implementation
test_node = Node('test')
print(test_node.data)

test


In [2]:
# Implement a linked list class
class LinkedList:
    
    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0
        
    def append(self, data):
        new_node = Node(data)
        
        if self.length == 0:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            new_node.prev = self.tail
            self.tail = new_node
        
        self.length += 1
        
    def __iter__(self):
        self._iter_node = self.head
        
        return self
    
    def __next__(self):
        if self._iter_node is None:
            raise StopIteration
        
        ret = self._iter_node.data
        self._iter_node = self._iter_node.next
        
        return ret
        
    def prepend(self, data):
        new_node = Node(data)
        
        if self.length == 0:
            self.head = new_node
            self.tail = new_node
        else:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
            
        self.length += 1
    
    # Enable linked list to work with in-built Python functions
    def __len__(self):
        return self.length
    
    def __str__(self):
        return str([value for value in self])
    
    
# Test the implementation
test_list = LinkedList()
test_list.append(1)
test_list.prepend(2)
test_list.prepend(3)

print(test_list)
print(len(test_list))

[3, 2, 1]
3


In [3]:
# Implement an Entry class to use within the dictionary. This is the key-value pair.
class Entry:
    
    def __init__(self, key, value):
        self.key = key
        self.value = value
        
    def __str__(self):
        return str(self.key)
                
# Implement a dictionary class
class Dictionary:
    
    def __init__(self, num_buckets):
        self.num_buckets = num_buckets
        self.buckets = [LinkedList() for _ in range(num_buckets)]
        self.length = 0
        
    def _get_index(self, key): # Returns the bucket index of a given key
        hash_code = hash(key)
        
        return hash_code % self.num_buckets
    
    def put(self, key, value): # Adds an entry
        index = self._get_index(key)
        
        found_key = False
        for entry in self.buckets[index]:
            if entry.key == key:
                entry.value = value
                found_key = True
        if not found_key:
            self.buckets[index].append(Entry(key, value))
            self.length += 1
            
    def get_value(self, key): # Retrieves a given key's value
        index = self._get_index(key)
        for entry in self.buckets[index]:
            if entry.key == key:
                return entry.value
        raise KeyError(key)
        
    def delete(self, key): # Removes an entry
        index = self._get_index(key)
        new_bucket = LinkedList()
        
        for entry in self.buckets[index]:
            if entry.key != key:
                new_bucket.append(entry)
        if new_bucket.length < self.buckets[index].length:
            self.length -= 1
        self.buckets[index] = new_bucket
        
    def minimum(self, return_key=False): # returns the key and value of the minimum value. If there is more than one entry with the minimum value, the first one encountered is returned.
        min_key = None
        min_value = None
        
        for bucket in self.buckets:
            for entry in bucket:
                current_value = self.get_value(entry.key)
                if min_value is None or current_value < min_value:
                    min_key = entry.key
                    min_value = current_value
        
        if return_key:            
            return (min_key, min_value)
        else:
            return min_value
        
    def maximum(self, return_key=False): # Same as above but for the maximum value
        max_key = None
        max_value = None
        
        for bucket in self.buckets:
            for entry in bucket:
                current_value = self.get_value(entry.key)
                if max_value is None or current_value > max_value:
                    max_key = entry.key
                    max_value = current_value
        
        if return_key:            
            return (max_key, max_value)
        else:
            return max_value
                
        
    # Enable dictionary to work with in-built Python functions
    def __getitem__(self, key):
        return self.get_value(key)
    
    def __setitem__(self, key, value):
        self.put(key, value)
        
    def __len__(self):
        return self.length
    
    def __iter__(self):
        self._iter_index = 0
        
        return self
    
    def __next__(self):
        if self._iter_index == self.num_buckets:
            raise StopIteration
        
        ret = []
        for entry in self.buckets[self._iter_index]:
            ret.append(entry.key)
        
        self._iter_index += 1
          
        return ret
             
    
        
# Test the implementation
test_dict = Dictionary(5) # Pass the number of desired buckets as an argument. It is a rule-of-thumb to use a prime number to ensure bucket indexes are utilised equally.
test_dict['first test'] = 'one'
test_dict['second test'] = 2

print(test_dict['first test'])

test_dict['first test'] = 'first'
print(test_dict['first test'])

test_dict.delete('first test')
print(len(test_dict))


one
first
1


In [4]:
# I have also attempted to make Dictionary objects iterable to allow us to return all keys within a dictionary one-by-one. I was able to get the __next__() method to return the keys from each bucket in the form of a list.
# This means that it is neccesarry to execute a double for loop if we wish to access each key individually rather than in a list.
# For the purposes of this project, this is sufficient. In the future I will look to improve this functionality further.

# Test the iteration functionality
iter_dict = Dictionary(5)

for i in range(10):
    iter_dict[i] = i

for linked_list in iter_dict:
    for entry in linked_list:
        print(entry)



0
5
1
6
2
7
3
8
4
9


## Explore the Data

Now we've created our `Dictionary` class, we can read the data from all CSV files from the `prices` folder into a `Dictionary` instance, where the stock symbols are the keys and the corresponding data are the values.
Let's do this and display the data that corresponds to the `aapl` stock symbol.

In [5]:
import pandas as pd
import os

# Calculate number of buckets to use. The load factor should ideally be below 0.75.
file_count = len(os.listdir('prices'))
number_of_buckets = file_count // 0.75
print(number_of_buckets)
    
    


746.0


In [6]:
# Read the CSV files from the prices folder into a Dictionary instance
stock_prices = Dictionary(751) # I've chosen 751 as it's the first prime number greater than the result above

for stock_sym in os.listdir('prices'):
    stock_data = pd.read_csv(os.path.join('prices', stock_sym))
    stock_sym = stock_sym.replace('.csv', '')
    
    stock_prices[stock_sym] = stock_data
    
print(stock_prices['aapl'])

            date       close        open        high         low     volume
0     2007-01-03   83.800002   86.289999   86.579999   81.899999  309579900
1     2007-01-04   85.659998   84.050001   85.949998   83.820003  211815100
2     2007-01-05   85.049997   85.770000   86.199997   84.400002  208685400
3     2007-01-08   85.470000   85.959998   86.529998   85.280003  199276700
4     2007-01-09   92.570003   86.450003   92.979999   85.150000  837324600
...          ...         ...         ...         ...         ...        ...
2585  2017-04-10  143.169998  143.600006  143.880005  142.899994   18473000
2586  2017-04-11  141.630005  142.940002  143.350006  140.059998   30275300
2587  2017-04-12  141.800003  141.600006  142.149994  141.009995   20238900
2588  2017-04-13  141.050003  141.910004  142.380005  141.050003   17652900
2589  2017-04-17  141.830002  141.479996  141.880005  140.869995   16424000

[2590 rows x 6 columns]


## Minimum and Maximum Average Closing Prices

Now that we've read in the data, we can use a Dictionary object to compute the following aggregates:

* The average closing price of each stock.
* The minimum average closing price over all stocks.
* The maximum average closing price over all stocks.

In [7]:
# Calculate a dictionary where the keys are the stock symbols and the values are the average closing price of that stock
avg_close_prices = Dictionary(751)

for stock_sym in os.listdir('prices'):
    stock_data = pd.read_csv(os.path.join('prices', stock_sym))
    stock_sym = stock_sym.replace('.csv', '')
    
    avg_close_prices[stock_sym] = stock_data['close'].mean(axis=0)

for bucket in avg_close_prices:
    for key in bucket:
        print(key, avg_close_prices[key])

    

carv 6.872226529729721
colm 53.72719691235514
egan 3.546579150579136
cfnb 13.825111963706616
cidm 1.9935057915057948
ffiv 86.294579173745
ebay 35.188856186486554
airm 46.918258888030955
cuba 7.75155212355212
evol 5.7018532818532774
amwd 34.320258717374536
becn 25.59151352046335
bosc 2.1880193050193038
cunb 15.99822393513515
anat 97.93825093397685
cash 32.26195366332041
actg 15.997490346718152
ceco 13.657787633204645
dspg 9.215841698069479
bkmu 7.30632432432434
alxn 97.1099267011583
cntf 2.6595637119691156
bgfv 13.156471046718117
arcw 5.706739867181469
evlv 3.9725907335907293
faro 34.87222776949802
cnty 4.444320463320463
bwina 23.124262511969086
fbss 15.228308892278005
adrd 22.51748262046331
cinf 42.25041697451733
daio 3.6515559845559764
feim 8.712000000000025
bbgi 5.33829729729731
acls 3.343806946718146
amsf 30.34488032162161
dlhc 1.8903745173745172
abax 34.57868337992275
ahpi 3.404389961389958
boom 20.597038611196922
essa 12.126070440047481
centa 10.959813017140611
anss 62.32520078146

In [8]:
# Calculate the minimum average closing price across all stocks

print(avg_close_prices.minimum(return_key=True)) # I've implemented a parameter to give the option for the key to be returned as a tuple.

('blfs', 0.8122763011583004)


In [9]:
# Calculate the maximum average closing price across all stocks

print(avg_close_prices.maximum(return_key=True))

('amzn', 275.1340775710431)


## Grouping Trades per Day

The Dictionary class we defined at the start of the project worked well and even incorporated functionality from Python's built-in dictionary class (such as: for loops, minimum value, maximum value, accessing and updating values using bracket notation).
However, as it is not as sophisticated as the built-in dictionary class, it was slow at performing some operations.

So, from this point onwards, we will use Python's built-in dictionary class. This will allow us to perform more complex computations and explore our data in more detail.

To answer the questions we are gonnna ask next, it will be convenient to have the trades organised by date. To do so, we'll calcultate a dictionary where the keys are the dates and the values are a list of all trades from all stock symbols that occured that day.

More precisely, for each day, we'll have a list of pairs `(volume, stock_symbol)`

In [10]:
# Recreate the stock_prices dictionary using Python's built-in class
import pandas as pd
import os

stock_prices = {}

for stock_sym in os.listdir('prices'):
    stock_data = pd.read_csv(os.path.join('prices', stock_sym))
    stock_sym = stock_sym.replace('.csv', '')
    
    stock_prices[stock_sym] = stock_data

In [11]:
# Create a dictionary where the keys are the dates and the values are lists of (volume, stock_symbol) pairs
daily_trades = {}

for stock_sym in stock_prices:
    for index, row in stock_prices[stock_sym].iterrows():
        date = row['date']
        volume = row['volume']
        pair = (volume, stock_sym)
        if date not in daily_trades:
            daily_trades[date] = []
        daily_trades[date].append(pair)

## Finding The Most Traded Stock Each Day

Now that we've created our dictionary, we can find the most traded stock for each day. To do this, we'll calculate a new dictionary where the keys are the days and the values are a tuple `trade_vol, stock_symbol`. The `stock_symbol` will represent the stock symbol that was most traded on that day and `trade_vol` represents the trade volume.

In [12]:
# Create a dictionary where the keys are the dates and the values are a tuple of (trade_vol, stock_symbol) for the most traded stock of that day.
daily_most_traded = {}

for date in daily_trades:
    daily_trades[date].sort()
    daily_most_traded[date] = daily_trades[date][-1]

# Test the implementation
print(daily_most_traded['2007-01-03'])
print(daily_most_traded['2007-01-04'])
print(daily_most_traded['2007-01-05'])
print(daily_most_traded['2007-01-08'])

(309579900, 'aapl')
(211815100, 'aapl')
(208685400, 'aapl')
(199276700, 'aapl')


## Searching For High Volume Days

Let's search for days that have unusually high volumes of trades. In order to do this, we will compute the total volume of trading for each day and then sort them to find which days come out on top.

We shall create a new dictionary where the keys are the dates and the values are the total `volume` of that day (over all stocks).

In [13]:
# Create a dictionary with dates as keys and the overall volume of trades as the values.
daily_total_volume = {}

for date in daily_trades:
    total_volume = sum([pair[0] for pair in daily_trades[date]])
    daily_total_volume[date] = total_volume

for date in sorted(daily_total_volume, key=daily_total_volume.get, reverse=True):
    print(date, ':', daily_total_volume[date])
    

2008-01-23 : 1964583900
2008-10-10 : 1770266900
2007-07-26 : 1611272800
2008-10-08 : 1599183500
2008-01-22 : 1578877700
2008-02-07 : 1559032100
2008-09-29 : 1555072400
2007-11-08 : 1553880500
2008-01-16 : 1536176400
2008-01-24 : 1533363200
2008-10-06 : 1504501200
2008-09-18 : 1474868300
2008-01-18 : 1443117700
2008-10-16 : 1432975400
2007-01-09 : 1416558400
2008-11-20 : 1409737200
2010-05-07 : 1408049500
2007-08-16 : 1379620500
2008-11-21 : 1373161300
2008-11-13 : 1372101300
2010-05-06 : 1367006700
2011-08-09 : 1353711300
2008-01-17 : 1348647200
2008-10-07 : 1332564500
2008-01-15 : 1331136400
2011-02-10 : 1323193700
2011-08-08 : 1315543500
2007-01-10 : 1315041800
2008-01-09 : 1311926600
2008-10-23 : 1311351400
2008-10-03 : 1309466400
2011-08-05 : 1298330500
2008-10-14 : 1296005000
2008-10-22 : 1280719200
2008-01-07 : 1270845600
2007-11-09 : 1267003100
2008-10-09 : 1258307600
2007-08-09 : 1254064200
2007-11-12 : 1241171300
2008-01-25 : 1241116300
2008-01-31 : 1239504500
2007-08-08 : 123

2007-06-28 : 759872300
2009-01-23 : 759750100
2011-07-20 : 759029100
2009-06-05 : 758989500
2010-05-18 : 758710600
2010-07-06 : 758624800
2007-05-01 : 758371300
2009-04-16 : 758216300
2009-10-23 : 758104900
2010-02-26 : 758085700
2016-11-10 : 758052300
2011-02-22 : 757839600
2007-04-30 : 757528300
2009-10-29 : 757238300
2008-06-25 : 756441200
2010-02-01 : 756031500
2007-02-14 : 755958900
2007-03-06 : 755488800
2007-06-14 : 754968500
2014-01-27 : 754944100
2009-06-17 : 754452700
2012-02-29 : 753844900
2007-10-01 : 753396700
2008-06-04 : 753044100
2008-09-02 : 753011100
2010-07-14 : 752755000
2010-02-03 : 752751000
2010-02-12 : 751438300
2011-10-06 : 751406900
2009-02-11 : 751122900
2010-02-02 : 751064200
2008-06-19 : 750867800
2009-03-09 : 750099200
2010-10-20 : 749981600
2013-01-23 : 749887800
2010-04-23 : 749641300
2011-09-29 : 749598700
2010-04-14 : 749470300
2012-03-15 : 749007300
2010-02-25 : 747794900
2010-08-24 : 747293400
2009-03-17 : 747178400
2007-08-28 : 747040800
2009-02-18 

2010-03-24 : 623811000
2012-02-17 : 623622400
2012-06-15 : 623535200
2009-08-28 : 623375600
2012-10-09 : 623339600
2012-02-10 : 622780000
2014-05-07 : 622658700
2009-11-24 : 622596900
2012-05-11 : 622319600
2009-12-16 : 622115900
2011-03-01 : 622060600
2013-02-08 : 621992100
2009-10-13 : 621746200
2009-08-24 : 621268700
2010-12-07 : 620731100
2007-08-24 : 620659300
2011-01-26 : 620322200
2013-04-19 : 620289400
2011-11-11 : 619757900
2011-09-15 : 619653200
2012-03-28 : 619014600
2010-11-05 : 618997100
2007-12-27 : 618836500
2009-09-14 : 618609700
2009-11-30 : 617659400
2014-03-13 : 617658700
2012-03-20 : 617148700
2013-02-01 : 616926400
2015-04-30 : 616806400
2012-12-07 : 616487600
2014-03-04 : 616299300
2011-09-12 : 616209100
2009-11-23 : 616017500
2012-11-26 : 615824500
2007-03-22 : 615219300
2010-03-26 : 615187400
2015-01-28 : 615039000
2015-01-30 : 614863000
2010-08-04 : 614776700
2012-11-30 : 614734700
2011-04-27 : 614664200
2008-12-30 : 614611000
2011-07-22 : 614608100
2008-08-19 

2014-03-10 : 508331100
2016-11-22 : 508271700
2017-04-04 : 508138800
2012-08-22 : 507903600
2008-07-03 : 507465100
2013-11-18 : 507391000
2015-03-25 : 507186800
2013-07-02 : 507163300
2014-12-15 : 507146500
2010-03-29 : 507127400
2015-11-03 : 507119600
2012-05-14 : 506843000
2011-12-15 : 506610900
2014-05-09 : 506578000
2013-12-12 : 506545200
2013-12-06 : 506458600
2013-09-04 : 506418500
2011-06-13 : 506104400
2012-08-08 : 505507700
2014-06-09 : 505136200
2014-07-01 : 505049900
2013-06-05 : 504979000
2015-11-13 : 504966100
2011-05-13 : 504853800
2014-02-10 : 504814000
2017-03-15 : 504812700
2013-10-02 : 504757100
2011-06-09 : 504692200
2008-12-29 : 504511400
2015-10-23 : 504234900
2011-04-04 : 504187700
2012-10-05 : 503907900
2016-09-14 : 503646800
2011-12-01 : 503550300
2010-04-12 : 503494900
2012-08-07 : 503411500
2013-06-07 : 503366000
2016-01-12 : 503066900
2010-08-30 : 502983600
2016-05-20 : 502981900
2016-09-15 : 502564500
2011-05-25 : 502550500
2015-05-01 : 502541200
2011-01-13 

2012-12-26 : 385176600
2016-11-28 : 384089100
2016-05-26 : 383356100
2016-08-17 : 383320200
2015-06-15 : 383090600
2017-03-23 : 382638900
2015-03-23 : 382584200
2016-07-15 : 382016600
2014-05-23 : 381868200
2016-07-18 : 381761000
2016-08-18 : 381616500
2014-06-26 : 381530300
2016-04-05 : 381126200
2017-01-23 : 381041500
2016-08-24 : 380892200
2016-05-09 : 380717900
2016-04-19 : 380499700
2016-06-03 : 380071600
2016-03-21 : 379641700
2016-06-10 : 379417600
2014-11-06 : 379120300
2015-04-14 : 379045900
2016-10-03 : 378491100
2016-08-09 : 378404000
2015-10-20 : 378336600
2015-06-17 : 378304300
2015-01-02 : 377538000
2016-10-05 : 377379300
2016-03-14 : 377287400
2016-05-13 : 375785600
2013-05-24 : 375778900
2014-10-06 : 375753500
2015-04-10 : 375645900
2013-09-13 : 375490500
2016-03-29 : 375172500
2016-09-19 : 375104800
2016-03-22 : 375085500
2012-12-28 : 374658100
2016-04-20 : 374268900
2013-12-30 : 374106600
2016-10-04 : 373989200
2013-08-23 : 373711300
2016-12-22 : 373530400
2016-04-12 

## Finding Profitable Stocks

Let's see which stocks would have been the most profitable to buy, from the first day of trading to the last day of trading in the data.
We can do this by:

* Subtracting the close price on the first day of trading from the close price of the last day of trading, then compute the percentage change.

* Sort all of the percentages

* Find the ten stocks that grew the most in the time period.

In [19]:
percentage_changes = []

for stock_sym in stock_prices:
    prices = stock_prices[stock_sym]
    start_close = prices.iloc[0, 1]
    end_close = prices.iloc[-1, 1]
    price_difference = end_close - start_close
    percentage_change = (price_difference / start_close) * 100
    percentage_changes.append((percentage_change, stock_sym))
                              
percentage_changes.sort(reverse=True)
                              
for result in percentage_changes[:10]:
    print(result)
    

(7483.8389225948395, 'admp')
(4005.0000000000005, 'adxs')
(3898.60048982856, 'arcw')
(2437.4365640858978, 'blfs')
(2230.7234281466817, 'amzn')
(1707.3554472785033, 'anip')
(1549.6700659868025, 'apdn')
(1525.1625162516252, 'cui')
(1339.2137535980346, 'bcli')
(1330.0000666666667, 'achc')


The most profitable stock to have bought in `2007` would have been `ADMP`, which had a 70 fold increase.